In [ ]:
from pyspark.streaming import StreamingContext
import sys

In [ ]:
outputPathPrefix="resSparkStreamingWinodws"

In [ ]:
# Create a Spark Streaming Context object
ssc = StreamingContext(sc, 5)

In [ ]:
# Set the checkpoint folder (it is needed by some window transformations)
ssc.checkpoint("checkpointfolder")

In [ ]:
# Create a (Receiver) DStream that will connect to localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

In [ ]:
# Apply the "standard" transformations to perform the word count task
# However, the "returned" RDDs are DStream RDDs
words = lines.flatMap(lambda line: line.split(" "))

In [ ]:
wordsOnes = words.map(lambda word: (word, 1))

In [ ]:
# reduceByKeyAndWindow is used instead of reduceByKey
# The characteristics of the window is also specified
wordsCounts = wordsOnes\
.reduceByKeyAndWindow(lambda v1, v2: v1+v2, None, 15)

In [ ]:
wordsCounts.pprint()

In [ ]:
wordsCounts.saveAsTextFiles(outputPathPrefix, "")

In [ ]:
#Start the computation
ssc.start()

In [ ]:
# Run this application for 90 seconds
ssc.awaitTerminationOrTimeout(90)
ssc.stop(stopSparkContext=False)